In [1]:
import numpy as np
import pandas as pd
from unidecode import unidecode
import re
from sklearn.datasets import load_files
import string
from underthesea import sent_tokenize
import unicodedata

d:\Code\Python Code\CT208\vm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chuyển từng câu thành cặp có dấu - không dấu, lưu vào file .csv

In [4]:
# bỏ đi số và dấu trong câu
def remove_punctuation_and_number(text):
    text = re.sub(r"[^a-zA-ZàáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ ]", " ", text)
    text = re.sub(r'\s+', " ", text)
    text = text.replace("nlđo", "").strip(" ")
    text = text.replace("NLĐO", "").strip(" ")
    return text

def preprocess_text(path):
    df = load_files(f"./{path}", encoding='utf-8')

    """
        - punctuation_sentences: tập các câu có dấu
        - non_punctuation_sentences: tập các câu không dấu
    """
    accented_sentences, unaccented_sentences = [], []
    for paragraph in df.data:
        # chuẩn hóa unicode
        paragraph = unicodedata.normalize("NFC", paragraph)
        # tách các câu trong bài báo
        sentences = sent_tokenize(paragraph)
        # chuyển câu sang lower case, bỏ đi khoảng trắng đầu, bỏ đi các dấu câu
        sentences = [remove_punctuation_and_number(sentence) for sentence in sentences]
        # bỏ đi các dấu thanh, chuyển từ câu có dấu -> câu không dấu
        new_sentences = [unidecode(sentence) for sentence in sentences]

        accented_sentences.extend(sentences)
        unaccented_sentences.extend(new_sentences)
    
    new_df = pd.DataFrame({'unaccented_sentences': unaccented_sentences, 'accented_sentences': accented_sentences})
    
    new_df.drop_duplicates(inplace=True)
    new_df.dropna(inplace=True)
    
    # lọc những câu có hơn 5 ký tự
    new_df['word_count'] = new_df['unaccented_sentences'].apply(lambda x: len(str(x).split()))
    new_df = new_df[new_df['word_count'] > 5]
    new_df = new_df.drop(columns=["word_count"])
    
    new_df.to_csv(f"{path}.csv", encoding="utf-8", index=False)

In [15]:
preprocess_text("train_set")
preprocess_text("val_set")
preprocess_text("test_set")

In [16]:
train_set = pd.read_csv("./train_set.csv").astype("str")
test_set = pd.read_csv("./test_set.csv").astype("str")
val_set = pd.read_csv("./val_set.csv").astype("str")

In [ ]:
train_set.info()
test_set.info()
val_set.info()

In [17]:
print(len(train_set))
print(len(val_set))
print(len(test_set))
print(len(train_set) + len(val_set) + len(test_set))

152270
6527
24740
183537


In [18]:
train = load_files("./train_set", encoding="utf-8")
test = load_files("./test_set", encoding="utf-8")
val = load_files("./val_set", encoding="utf-8")

print(len(train.data))
print(len(train.data) + len(test.data) + len(val.data))

9100
10768


In [19]:
print(len(test.data))
print(len(val.data))

1288
380
